In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('synthetic_wtp_laptop_data.csv')

print("=== DATASET OVERVIEW ===")
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
print(df.head())

# Define base model and upgrade costs
base_model = {
    'Memory': 16,
    'Storage': 512, 
    'CPU_class': 1,
    'Screen_size': 14.0,
    'year': 2025
}

upgrade_costs = {
    'memory': 7000,    
    'storage': 5000,   
    'cpu': 15000,     
    'screen': 3000     
}

current_selling_price = 111000

print(f"\n=== BASELINE ANALYSIS ===")
print(f"Base model specs: {base_model}")
print(f"Current selling price: {current_selling_price:,} yen")

# Find current model in market data
base_matches = df[
    (df['Memory'] == base_model['Memory']) &
    (df['Storage'] == base_model['Storage']) &
    (df['CPU_class'] == base_model['CPU_class']) &
    (df['Screen_size'] == base_model['Screen_size']) &
    (df['year'] == base_model['year'])
]

if len(base_matches) > 0:
    avg_market_price = base_matches['price'].mean()
    print(f"Market data matches: {len(base_matches)}")
    print(f"Average market price: {avg_market_price:,.0f} yen")
    print(f"Price difference: {avg_market_price - current_selling_price:+.0f} yen")
else:
    print("No exact matches found in market data")

# Define upgraded models
upgraded_models = {
    'memory': {**base_model, 'Memory': 32},
    'storage': {**base_model, 'Storage': 1024},
    'cpu': {**base_model, 'CPU_class': 2},
    'screen': {**base_model, 'Screen_size': 16}
}

print(f"\n=== UPGRADE ANALYSIS ===")

results = {}

# Analyze each upgrade option
for upgrade_type, specs in upgraded_models.items():
    print(f"\n--- {upgrade_type.upper()} UPGRADE ---")
    print(f"New specs: {specs}")
    
    # Find market matches for upgraded specs
    matches = df[
        (df['Memory'] == specs['Memory']) &
        (df['Storage'] == specs['Storage']) &
        (df['CPU_class'] == specs['CPU_class']) &
        (df['Screen_size'] == specs['Screen_size']) &
        (df['year'] == specs['year'])
    ]
    
    if len(matches) > 0:
        market_price = matches['price'].mean()
        upgrade_cost = upgrade_costs[upgrade_type]
        additional_revenue = market_price - current_selling_price
        gross_profit = additional_revenue - upgrade_cost
        
        print(f"Market matches: {len(matches)}")
        print(f"Average market price: {market_price:,.0f} yen")
        print(f"Upgrade cost: {upgrade_cost:,} yen")
        print(f"Additional revenue: {additional_revenue:+,.0f} yen")
        print(f"Gross profit: {gross_profit:+,.0f} yen")
        
        results[upgrade_type] = {
            'market_price': market_price,
            'upgrade_cost': upgrade_cost,
            'additional_revenue': additional_revenue,
            'gross_profit': gross_profit
        }
    else:
        print("No direct market matches found")
        results[upgrade_type] = None

# Handle CPU upgrade estimation if no direct matches
if results['cpu'] is None:
    print(f"\n--- CPU UPGRADE ESTIMATION ---")
    # Analyze CPU pricing patterns for similar configs
    cpu_analysis = {}
    for cpu_class in [0, 1, 2, 3]:
        cpu_matches = df[
            (df['Memory'] == 16) &
            (df['Storage'] == 512) &
            (df['CPU_class'] == cpu_class) &
            (df['year'] == 2025)
        ]
        if len(cpu_matches) > 0:
            cpu_analysis[cpu_class] = cpu_matches['price'].mean()
            print(f"CPU Class {cpu_class}: {len(cpu_matches)} matches, avg price: {cpu_analysis[cpu_class]:,.0f} yen")
    
    # Estimate CPU=2 pricing using price difference approach
    if 1 in cpu_analysis and 2 in cpu_analysis:
        # Use price difference between CPU classes to estimate our specific config
        price_difference = cpu_analysis[2] - cpu_analysis[1]
        estimated_price = current_selling_price + price_difference
        print(f"Price difference for CPU upgrade: +{price_difference:,.0f} yen")
        
        upgrade_cost = upgrade_costs['cpu']
        additional_revenue = estimated_price - current_selling_price
        gross_profit = additional_revenue - upgrade_cost
        
        print(f"Estimated market price for CPU upgrade: {estimated_price:,.0f} yen")
        print(f"Estimated gross profit: {gross_profit:+,.0f} yen")
        
        results['cpu'] = {
            'market_price': estimated_price,
            'upgrade_cost': upgrade_cost,
            'additional_revenue': additional_revenue,
            'gross_profit': gross_profit
        }

# Rank upgrades by gross profit
print(f"\n=== FINAL RANKING ===")
valid_results = {k: v for k, v in results.items() if v is not None}
ranked_upgrades = sorted(valid_results.items(), 
                        key=lambda x: x[1]['gross_profit'], 
                        reverse=True)

for i, (upgrade, data) in enumerate(ranked_upgrades, 1):
    print(f"{i}. {upgrade.upper()} upgrade")
    print(f"   Gross profit: {data['gross_profit']:+,.0f} yen")
    print(f"   (Revenue: +{data['additional_revenue']:,.0f} yen, Cost: {data['upgrade_cost']:,} yen)")

print(f"\n=== RECOMMENDATION ===")
print(f"TOP 2 UPGRADES for highest gross profit:")
if len(ranked_upgrades) >= 2:
    print(f"1. {ranked_upgrades[0][0].upper()} upgrade ({ranked_upgrades[0][1]['gross_profit']:+,.0f} yen)")
    print(f"2. {ranked_upgrades[1][0].upper()} upgrade ({ranked_upgrades[1][1]['gross_profit']:+,.0f} yen)")
else:
    print("Insufficient data for full ranking")

=== DATASET OVERVIEW ===
Dataset shape: (1000, 6)
Columns: ['Memory', 'Storage', 'CPU_class', 'Screen_size', 'year', 'price']

First few rows:
   Memory  Storage  CPU_class  Screen_size  year   price
0       8     1024          3         16.0  2025  162021
1      32     1024          0         16.0  2024  112214
2      16      512          1         16.0  2025  118354
3      16     1024          1         16.0  2025  123383
4      16      256          3         16.0  2024  162968

=== BASELINE ANALYSIS ===
Base model specs: {'Memory': 16, 'Storage': 512, 'CPU_class': 1, 'Screen_size': 14.0, 'year': 2025}
Current selling price: 111,000 yen
Market data matches: 6
Average market price: 111,054 yen
Price difference: +54 yen

=== UPGRADE ANALYSIS ===

--- MEMORY UPGRADE ---
New specs: {'Memory': 32, 'Storage': 512, 'CPU_class': 1, 'Screen_size': 14.0, 'year': 2025}
Market matches: 3
Average market price: 130,513 yen
Upgrade cost: 7,000 yen
Additional revenue: +19,513 yen
Gross profit: +12,5